In [ ]:
from datetime import date
print(date.today())

# Normalisation and variance stabilisation



# 1. Normalisation

Objectives: 

1. Explain rationale behind normalisation/variance stabilisation
1. List options for normalisation/variance stabilisation
1. Explain their preference for one
1. Normalise a dataset
1. Demonstrate that the normalisation achieved the intended effect

### 1.1 Why is normalisation important?

- have students run without normalisation and see what happens.
- does normalisation distort/enhance biological signal?
- literature about normalisation:
    - [Comparison of transformations for single-cell RNA-seq data](https://www.nature.com/articles/s41592-023-01814-1)
    - [Validation of noise models for single-cell transcriptomics](https://www.nature.com/articles/nmeth.2930)
    - [The triumphs and limitations of computational methods for scRNA-seq](https://www.nature.com/articles/s41592-021-01171-x)

### 1.2 How do we normalise?

- challenge them to think about what we need. What does it mean if we normalise?
- What do we want to achieve with normalisation? Do we get it?
- Literature:
    - [Normalisation of single-cell RNA-seq counts by log(x+1)](https://academic.oup.com/bioinformatics/article/37/15/2223/6155989)
    - [Normalization and variance stabilization of single-cell RNA-seq data using regularized negative binomial regression](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1874-1)
    - [Depth normalization for single-cell genomics count data](https://www.biorxiv.org/content/10.1101/2022.05.06.490859v1.full)

### 1.3 How do we evaluate if it worked?

- gold standard
- simulated data
- expert knowledge